# 🔄 Week 3-4: Cross Validation (교차 검증) 마스터!

## 🤔 지금까지의 문제점
**한 번만 나누기의 한계**:
- 지금까지 `train_test_split`으로 **딱 한 번만** 나눴음
- 운이 좋으면 → 높은 점수
- 운이 나쁘면 → 낮은 점수
- **"진짜 실력"을 알기 어려움**

## 🎯 Cross Validation의 해결책
**여러 번 나눠서 평균내기**:
- 데이터를 **5번 다르게** 나눠서 테스트
- 5번의 점수를 **평균**내서 최종 점수
- **더 정확하고 신뢰할 수 있는** 평가!

## 📚 오늘 배울 내용
1. **Cross Validation 개념** 완전 이해
2. **K-Fold Cross Validation** 실습
3. **분류와 회귀** 모두 적용해보기
4. **진짜 실력** 측정하는 방법

---


## 📖 Cross Validation 개념 완전 이해하기

### 🔸 **기존 방식 (train_test_split)**:
```
전체 데이터 100개
├── 훈련용 80개 (학습)
└── 테스트용 20개 (평가) → 점수 1개
```

### 🔄 **Cross Validation 방식 (5-Fold)**:
```
전체 데이터 100개를 5등분
라운드 1: [테스트20] [훈련80개] → 점수1
라운드 2: [훈련20] [테스트20] [훈련60개] → 점수2  
라운드 3: [훈련40] [테스트20] [훈련40개] → 점수3
라운드 4: [훈련60] [테스트20] [훈련20개] → 점수4
라운드 5: [훈련80] [테스트20개] → 점수5

최종 점수 = (점수1 + 점수2 + 점수3 + 점수4 + 점수5) / 5
```

### 💡 **왜 더 좋은가?**
- **운에 의존하지 않음** (여러 번 테스트)
- **모든 데이터가 테스트에 참여** (공정함)
- **표준편차까지 알 수 있음** (안정성 측정)

### 🎯 **언제 사용할까?**
- **모델 선택** (어떤 알고리즘이 최고?)
- **하이퍼파라미터 튜닝** (최적 설정 찾기)
- **진짜 성능 확인** (과적합 체크)


In [25]:
# 🛠️ 도구 준비
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

print("🔧 Cross Validation 도구 준비 완료!")
print("📊 오늘의 무기들:")
print("   🔄 cross_val_score - 교차 검증 실행")
print("   📁 KFold - 데이터 분할 전략")
print("   🌳 분류 알고리즘들 (Decision Tree, Random Forest, SVM)")
print("   📈 회귀 알고리즘들 (Linear, Ridge, Lasso)")
print("   📏 평가 지표들 (accuracy, MSE, R²)")

print("\n🎯 목표: 진짜 실력을 측정해보자!")


🔧 Cross Validation 도구 준비 완료!
📊 오늘의 무기들:
   🔄 cross_val_score - 교차 검증 실행
   📁 KFold - 데이터 분할 전략
   🌳 분류 알고리즘들 (Decision Tree, Random Forest, SVM)
   📈 회귀 알고리즘들 (Linear, Ridge, Lasso)
   📏 평가 지표들 (accuracy, MSE, R²)

🎯 목표: 진짜 실력을 측정해보자!


## 🌸 실습 1: 분류에서 Cross Validation

**분류 예제**: 아이리스 꽃 분류
- 기존 방식 vs Cross Validation 비교
- 어떤 차이가 있는지 직접 확인!


In [26]:
# 🌸 아이리스 데이터 준비
iris = datasets.load_iris()
X_iris, y_iris = iris.data, iris.target

print("🌸 아이리스 데이터 정보:")
print(f"   🌺 총 꽃 개수: {X_iris.shape[0]}개")
print(f"   📊 특성 개수: {X_iris.shape[1]}개") 
print(f"   🏷️ 꽃 종류: {len(np.unique(y_iris))}가지")
print(f"   📝 종류명: {iris.target_names}")

print("\n🎯 목표: 3개 분류 알고리즘의 진짜 실력 비교!")


🌸 아이리스 데이터 정보:
   🌺 총 꽃 개수: 150개
   📊 특성 개수: 4개
   🏷️ 꽃 종류: 3가지
   📝 종류명: ['setosa' 'versicolor' 'virginica']

🎯 목표: 3개 분류 알고리즘의 진짜 실력 비교!


### 🔸 방법 1: 기존 방식 (한 번만 나누기)


In [27]:
# 🔸 기존 방식: train_test_split 한 번만
X_train, X_test, y_train, y_test = train_test_split(
    X_iris, y_iris, test_size=0.3, random_state=42
)

# 3개 분류 알고리즘 준비
algorithms = {
    'Decision Tree 🌳': DecisionTreeClassifier(random_state=42),
    'Random Forest 🌲': RandomForestClassifier(random_state=42),
    'SVM ⚔️': SVC(random_state=42)
}

print("🔸 기존 방식 결과 (한 번만 분할):")
print("="*50)

traditional_scores = {}
for name, model in algorithms.items():
    # 훈련 및 예측
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    traditional_scores[name] = score
    
    print(f"{name}: {score:.3f} ({score*100:.1f}%)")

print(f"\n📊 기존 방식 - 최고 성능: {max(traditional_scores.values()):.3f}")
print("⚠️  하지만 이게 진짜 실력일까? 운이 좋았을 수도...")


🔸 기존 방식 결과 (한 번만 분할):
Decision Tree 🌳: 1.000 (100.0%)
Random Forest 🌲: 1.000 (100.0%)
SVM ⚔️: 1.000 (100.0%)

📊 기존 방식 - 최고 성능: 1.000
⚠️  하지만 이게 진짜 실력일까? 운이 좋았을 수도...


### 🔄 방법 2: Cross Validation (5번 나누기)


In [28]:
# 🔄 Cross Validation: 5번 나누기
print("🔄 Cross Validation 결과 (5번 분할 후 평균):")
print("="*60)

cv_results = {}
for name, model in algorithms.items():
    # 5-Fold Cross Validation
    cv_scores = cross_val_score(model, X_iris, y_iris, cv=5, scoring='accuracy')
    
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    cv_results[name] = {'mean': cv_mean, 'std': cv_std, 'scores': cv_scores}
    
    print(f"\n{name}:")
    print(f"   📊 5번 점수: {[f'{score:.3f}' for score in cv_scores]}")
    print(f"   📈 평균: {cv_mean:.3f} ({cv_mean*100:.1f}%)")
    print(f"   📏 표준편차: {cv_std:.3f} (안정성 지표)")
    
    # 기존 방식과 비교
    traditional = traditional_scores[name]
    diff = cv_mean - traditional
    if abs(diff) > 0.01:
        direction = "높아짐" if diff > 0 else "낮아짐"
        print(f"   🔄 기존 대비: {diff:+.3f} ({direction})")
    else:
        print(f"   🔄 기존과 비슷: 차이 {diff:+.3f}")

# 최고 성능 알고리즘 찾기
best_algorithm = max(cv_results.keys(), key=lambda x: cv_results[x]['mean'])
best_score = cv_results[best_algorithm]['mean']
best_std = cv_results[best_algorithm]['std']

print(f"\n🏆 Cross Validation 최고 성능:")
print(f"   🥇 우승자: {best_algorithm}")
print(f"   📊 성능: {best_score:.3f} ± {best_std:.3f}")
print(f"   💡 해석: 95% 확률로 {(best_score-2*best_std)*100:.1f}%~{(best_score+2*best_std)*100:.1f}% 사이 성능")


🔄 Cross Validation 결과 (5번 분할 후 평균):

Decision Tree 🌳:
   📊 5번 점수: ['0.967', '0.967', '0.900', '0.933', '1.000']
   📈 평균: 0.953 (95.3%)
   📏 표준편차: 0.034 (안정성 지표)
   🔄 기존 대비: -0.047 (낮아짐)

Random Forest 🌲:
   📊 5번 점수: ['0.967', '0.967', '0.933', '0.967', '1.000']
   📈 평균: 0.967 (96.7%)
   📏 표준편차: 0.021 (안정성 지표)
   🔄 기존 대비: -0.033 (낮아짐)

SVM ⚔️:
   📊 5번 점수: ['0.967', '0.967', '0.967', '0.933', '1.000']
   📈 평균: 0.967 (96.7%)
   📏 표준편차: 0.021 (안정성 지표)
   🔄 기존 대비: -0.033 (낮아짐)

🏆 Cross Validation 최고 성능:
   🥇 우승자: Random Forest 🌲
   📊 성능: 0.967 ± 0.021
   💡 해석: 95% 확률로 92.5%~100.9% 사이 성능


## 🏠 실습 2: 회귀에서 Cross Validation

**회귀 예제**: 캘리포니아 집값 예측
- 회귀에서는 `R²` 점수로 평가
- 분류와 어떤 차이가 있는지 확인!


In [29]:
# 🏠 캘리포니아 집값 데이터 준비
california = datasets.fetch_california_housing()
X_house, y_house = california.data, california.target

print("🏠 캘리포니아 집값 데이터 정보:")
print(f"   🏘️ 총 집 개수: {X_house.shape[0]:,}개")
print(f"   📊 특성 개수: {X_house.shape[1]}개")
print(f"   💰 집값 범위: ${y_house.min():.1f}만 ~ ${y_house.max():.1f}만")
print(f"   💵 평균 집값: ${y_house.mean():.1f}만")

print("\n🎯 목표: 3개 회귀 알고리즘의 진짜 실력 비교!")


🏠 캘리포니아 집값 데이터 정보:
   🏘️ 총 집 개수: 20,640개
   📊 특성 개수: 8개
   💰 집값 범위: $0.1만 ~ $5.0만
   💵 평균 집값: $2.1만

🎯 목표: 3개 회귀 알고리즘의 진짜 실력 비교!


In [30]:
# 🏠 회귀 알고리즘들로 Cross Validation
regression_algorithms = {
    'Linear 🏃‍♂️': LinearRegression(),
    'Ridge 🛡️': Ridge(alpha=1.0, random_state=42),
    'Lasso ✂️': Lasso(alpha=0.1, random_state=42)
}

print("🏠 회귀 Cross Validation 결과 (R² Score 기준):")
print("="*60)

regression_cv_results = {}
for name, model in regression_algorithms.items():
    # 5-Fold Cross Validation (R² Score 사용)
    cv_scores = cross_val_score(model, X_house, y_house, cv=5, scoring='r2')
    
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    regression_cv_results[name] = {'mean': cv_mean, 'std': cv_std, 'scores': cv_scores}
    
    print(f"\n{name}:")
    print(f"   📊 5번 R² 점수: {[f'{score:.3f}' for score in cv_scores]}")
    print(f"   📈 평균 R²: {cv_mean:.3f} ({cv_mean*100:.1f}% 설명력)")
    print(f"   📏 표준편차: {cv_std:.3f}")
    
    # 안정성 평가
    if cv_std < 0.01:
        stability = "매우 안정적 🟢"
    elif cv_std < 0.05:
        stability = "안정적 🟡"  
    else:
        stability = "불안정 🔴"
    print(f"   🎯 안정성: {stability}")

# 최고 성능 회귀 알고리즘
best_regression = max(regression_cv_results.keys(), key=lambda x: regression_cv_results[x]['mean'])
best_r2 = regression_cv_results[best_regression]['mean']
best_r2_std = regression_cv_results[best_regression]['std']

print(f"\n🏆 회귀 Cross Validation 최고 성능:")
print(f"   🥇 우승자: {best_regression}")
print(f"   📊 R² 성능: {best_r2:.3f} ± {best_r2_std:.3f}")
print(f"   💡 해석: {best_r2*100:.1f}% 설명력, 95% 확률로 {(best_r2-2*best_r2_std)*100:.1f}%~{(best_r2+2*best_r2_std)*100:.1f}% 사이")


🏠 회귀 Cross Validation 결과 (R² Score 기준):

Linear 🏃‍♂️:
   📊 5번 R² 점수: ['0.549', '0.468', '0.551', '0.537', '0.661']
   📈 평균 R²: 0.553 (55.3% 설명력)
   📏 표준편차: 0.062
   🎯 안정성: 불안정 🔴

Ridge 🛡️:
   📊 5번 R² 점수: ['0.549', '0.468', '0.551', '0.537', '0.661']
   📈 평균 R²: 0.553 (55.3% 설명력)
   📏 표준편차: 0.062
   🎯 안정성: 불안정 🔴

Lasso ✂️:
   📊 5번 R² 점수: ['0.513', '0.429', '0.511', '0.388', '0.520']
   📈 평균 R²: 0.472 (47.2% 설명력)
   📏 표준편차: 0.054
   🎯 안정성: 불안정 🔴

🏆 회귀 Cross Validation 최고 성능:
   🥇 우승자: Ridge 🛡️
   📊 R² 성능: 0.553 ± 0.062
   💡 해석: 55.3% 설명력, 95% 확률로 43.0%~67.6% 사이


## 📊 분류 vs 회귀 Cross Validation 비교

**두 가지 방식의 차이점을 정리해보자!**


In [31]:
# 📊 최종 비교 및 정리
print("📊 Cross Validation 최종 정리:")
print("="*70)

print("\n🌸 분류 (아이리스 꽃):")
for name, results in cv_results.items():
    mean_score = results['mean']
    std_score = results['std']
    print(f"   {name}: {mean_score:.3f} ± {std_score:.3f} ({mean_score*100:.1f}% 정확도)")

print(f"\n🏆 분류 우승자: {best_algorithm} ({best_score:.3f})")

print("\n🏠 회귀 (캘리포니아 집값):")
for name, results in regression_cv_results.items():
    mean_r2 = results['mean'] 
    std_r2 = results['std']
    print(f"   {name}: {mean_r2:.3f} ± {std_r2:.3f} ({mean_r2*100:.1f}% 설명력)")

print(f"\n🏆 회귀 우승자: {best_regression} ({best_r2:.3f})")

print("\n💡 Cross Validation의 핵심 가치:")
print("   ✅ 운에 의존하지 않는 정확한 성능 측정")
print("   ✅ 모든 데이터가 테스트에 참여하는 공정성")
print("   ✅ 표준편차로 안정성까지 확인 가능")
print("   ✅ 과적합 여부를 더 정확히 판단")

print("\n🎯 언제 사용하면 좋을까?")
print("   📈 모델 선택 시 (어떤 알고리즘이 최고인지)")
print("   🔧 하이퍼파라미터 튜닝 시")
print("   🏆 최종 성능 보고 시")
print("   ⚠️  데이터가 적을 때 (더 많은 정보 활용)")


📊 Cross Validation 최종 정리:

🌸 분류 (아이리스 꽃):
   Decision Tree 🌳: 0.953 ± 0.034 (95.3% 정확도)
   Random Forest 🌲: 0.967 ± 0.021 (96.7% 정확도)
   SVM ⚔️: 0.967 ± 0.021 (96.7% 정확도)

🏆 분류 우승자: Random Forest 🌲 (0.967)

🏠 회귀 (캘리포니아 집값):
   Linear 🏃‍♂️: 0.553 ± 0.062 (55.3% 설명력)
   Ridge 🛡️: 0.553 ± 0.062 (55.3% 설명력)
   Lasso ✂️: 0.472 ± 0.054 (47.2% 설명력)

🏆 회귀 우승자: Ridge 🛡️ (0.553)

💡 Cross Validation의 핵심 가치:
   ✅ 운에 의존하지 않는 정확한 성능 측정
   ✅ 모든 데이터가 테스트에 참여하는 공정성
   ✅ 표준편차로 안정성까지 확인 가능
   ✅ 과적합 여부를 더 정확히 판단

🎯 언제 사용하면 좋을까?
   📈 모델 선택 시 (어떤 알고리즘이 최고인지)
   🔧 하이퍼파라미터 튜닝 시
   🏆 최종 성능 보고 시
   ⚠️  데이터가 적을 때 (더 많은 정보 활용)


## 🎓 Cross Validation 마스터 완료!

**축하합니다! 이제 진짜 AI 성능 측정 전문가가 되었어요!** 🏆

### ✅ **오늘 배운 핵심 내용**:
1. **Cross Validation 개념** - 여러 번 나눠서 평균내기
2. **K-Fold 방식** - 5번 나누어 더 정확한 평가
3. **분류와 회귀 적용** - accuracy vs R² Score
4. **안정성 측정** - 표준편차로 일관성 확인
5. **실무 활용법** - 언제, 어떻게 사용할지

### 🚀 **다음 단계**:
- **혼동행렬과 평가지표** (정밀도, 재현율, F1-Score)
- **Week 4: 타이타닉 생존 예측** 첫 ML 프로젝트
- **하이퍼파라미터 튜닝** (GridSearchCV)

**이제 어떤 AI 모델이든 제대로 평가할 수 있어요!** ✨
